In [2]:
import sagemaker 
from sagemaker import get_execution_role
import boto3
import pandas as pd
import io

In [5]:

s3 = boto3.client('s3')
bucket_name = 'bucketgrupo2'
response = s3.get_object(Bucket='bucketgrupo2', Key='dataset_processado.csv')
data = response['Body'].read()

In [6]:
df = pd.read_csv(io.StringIO(data.decode('utf-8')))

In [26]:
df

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,no_of_special_requests,...,arrival_date_23,arrival_date_24,arrival_date_25,arrival_date_26,arrival_date_27,arrival_date_28,arrival_date_29,arrival_date_30,arrival_date_31,label_avg_price_per_room
0,2,0,1,2,0,0.505643,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,2,3,0,0.011287,0,0,0.0,1,...,0,0,0,0,0,0,0,0,0,1
2,1,0,2,1,0,0.002257,0,0,0.0,0,...,0,0,0,0,0,1,0,0,0,0
3,2,0,0,2,0,0.476298,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,1
4,2,0,1,1,0,0.108352,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,3,0,2,6,0,0.191874,0,0,0.0,1,...,0,0,0,0,0,0,0,0,0,2
36271,2,0,1,3,0,0.514673,0,0,0.0,2,...,0,0,0,0,0,0,0,0,0,1
36272,2,0,2,6,0,0.334086,0,0,0.0,2,...,0,0,0,0,0,0,0,0,0,1
36273,2,0,0,3,0,0.142212,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint


X = df.drop("label_avg_price_per_room", axis=1)
y = df["label_avg_price_per_room"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#y_test = keras.utils.to_categorical(y_test, num_classes=3)
#y_train = keras.utils.to_categorical(y_train, num_classes=3)

model = keras.Sequential()
model.add(layers.Dense(73, input_shape=(X.shape[1],), activation='relu'))
model.add(layers.Dense(37, activation='relu'))
model.add(Dropout(0.3))
model.add(layers.Dense(37, activation='relu'))
model.add(Dropout(0.3))
model.add(layers.Dense(37, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))


model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=100,batch_size=64,
                    validation_data=(X_test, y_test))

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print("Test accuracy:", test_acc)

Epoch 1/100
Extension horovod.torch has not been built: /usr/local/lib/python3.8/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-38-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2023-01-30 01:02:23.574 tensorflow-2-6-cpu-py-ml-t3-medium-9169b2e75617c45c79c40579f6a8:656 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-01-30 01:02:23.631 tensorflow-2-6-cpu-py-ml-t3-medium-9169b2e75617c45c79c40579f6a8:656 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
454/454 [==============================] - 3s 4ms/step - loss: 0.8411 - accuracy: 0.5988 - val_loss: 0.6778 - val_accuracy: 0.7019
Epoch 2/100
454/454 [==============================] - 2s 4ms/step - loss: 0.6730 - accuracy: 0.7049 - val_loss: 0.6130 - val_accuracy: 0.7336
Epoch 3/100
454/454 [=

In [8]:
model.save('modelo.h5')

In [29]:
with open('modelo.h5', 'rb') as data:
    s3.upload_fileobj(data, bucket_name, 'modelo.h5')